# Мемы

In [101]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [102]:
import pandas as pd
import numpy as np

In [103]:
path = 'drive/MyDrive/DS/meme'


In [104]:
# event = pd.read_csv(path+'/danila_event.csv')
# meme = pd.read_csv(path+'/danila_meme.csv')
# user = pd.read_csv(path+'/danila_user.csv')
event = pd.read_csv(path+'/tgbot_event.csv')
meme = pd.read_csv(path+'/tgbot_meme.csv')
user = pd.read_csv(path+'/tgbot_user.csv')

In [193]:
meme.head()

,meme_id,meme_type,original_post_link,source_name,published_at,telegram_file_id,content_url,caption,content_lang,is_available,likes,dislikes,created_at,updated_at,is_broken,source_id,content_text,caption_lang,language_id
0,13395,photo,https://vk.com/saintbeobanka?w=wall-67185996_2...,https://vk.com/saintbeobanka,2020-03-28 00:22:26+00,AgACAgQAAx0EVqG9GwACyDJefsuwVgIydYSqXSt9L2-o3Y...,https://sun6-14.userapi.com/c7004/v7004381/823...,NaN,ru,True,28,38,2020-03-28 03:59:44.496141+00,2021-02-10 07:21:58.272992+00,False,10.0,Будильник Отложить на 10 мимут vk.com/jaimes_h...,ww,2.0
1,13952,photo,https://reddit.com/r/wholesomememes/comments/f...,https://www.reddit.com/r/wholesomememes/,2020-03-28 12:14:46+00,AgACAgQAAx0EVqG9GwAC0QZef4L5HUrdey1K4xOBNC0V0-...,https://i.redd.it/xo8ilxurnep41.png,Wholesome slavery?,en,True,9,16,2020-03-28 17:01:45.289559+00,2021-02-10 07:21:58.276687+00,False,80.0,Happy Thanksgiving ! I was gonna say Let's say...,en,3.0
2,15593,photo,https://t.me/LaQeque/41396,https://t.me/LaQeque,2020-03-30 09:11:04+00,AgACAgIAAx0CVqG9GwAC7X5egk_f1FcUOOXdwvQ6tCUBOP...,https://storage.googleapis.com/meme_imgs/15593...,NaN,ru,True,41,54,2020-03-30 12:02:40.780487+00,2021-02-10 07:21:58.28043+00,False,19.0,"Ггграждане, мемння ззовут вваш преззидент, ппо...",ww,2.0
3,18442,photo,https://reddit.com/r/dankmemes/comments/ftalgn...,https://www.reddit.com/r/dankmemes/,2020-04-01 22:53:31+00,AgACAgQAAx0EVqG9GwABAQ8_XoVyYdS-2R9bcIpdH4YqK-...,https://i.redd.it/a3mp5spddaq41.jpg,You Won’t last 10 seconds,en,True,87,48,2020-04-02 05:04:33.300689+00,2021-02-10 07:21:58.28401+00,False,78.0,AL ALL WT Should we contact the hot milfs in o...,en,3.0
4,20798,photo,https://vk.com/wtf.rasha?w=wall-23486067_179559,https://vk.com/wtf.rasha,2020-04-04 11:15:01+00,AgACAgQAAx0EVqG9GwABATuvXonKXWZuSeRqN8vd8gVR09...,https://sun9-37.userapi.com/c858228/v858228646...,NaN,ru,True,234,178,2020-04-04 12:10:28.173755+00,2021-02-10 07:21:58.291519+00,False,9.0,Шляпа Патриарха Никона @NikonaShapka апрель 20...,ww,2.0


In [106]:
user.head()

,user_id,username,first_name,last_name,language_code,seed,deep_link,is_blocked_bot,is_banned,is_admin,is_moderator,sessions,last_session_memes,watched_memes,created_at,updated_at,interface_lang,gender,latest_meme_sent_at
0,1235209430,NaN,Paras,Kamboj,NaN,6,likemeupbot_lang_en,True,False,False,0,3,4,8,2020-10-27 07:33:12.040495+00,2021-02-10 00:29:57.669801+00,en,NaN,NaN
1,113584040,nikrnrr,N,NaN,ru,26,NaN,True,False,False,0,1,1,0,2020-10-28 12:05:37.473558+00,2021-02-10 01:29:57.505289+00,ru,N,NaN
2,1142713343,NaN,Serik,NaN,NaN,32,NaN,True,False,False,0,1,1,0,2020-10-27 17:31:30.480084+00,2021-02-10 01:29:55.555919+00,en,NaN,NaN
3,187967374,Ddivanov,Dmitry,Ivanov,ru,13,NaN,True,False,False,0,2,5,9,2020-10-26 13:12:52.373779+00,2021-02-09 23:29:56.086916+00,ru,M,NaN
4,923016092,NaN,Joanna,NaN,es,99,NaN,False,False,False,0,39,1,35,2020-10-27 19:11:33.959974+00,2021-02-09 17:56:04.574689+00,en,NaN,2021-02-09 17:56:04.574558+00


In [107]:
event.head()

,event_id,reaction_id,session_idx,meme_idx,created_at,reacted_at,meme_id,user_id,telegram_message_id
0,1569409,1.0,1.0,171.0,2020-09-23 14:45:40.204598+00,2020-09-23 14:45:44.079206+00,9197,1153946021,1624299.0
1,1569228,2.0,199.0,189.0,2020-09-23 14:38:56.25774+00,2020-09-23 14:39:11.978709+00,34459,285739883,1624096.0
2,1569405,1.0,7.0,133.0,2020-09-23 14:45:34.244269+00,2020-09-23 14:45:46.699976+00,10879,751332478,1624295.0
3,1569239,1.0,2.0,265.0,2020-09-23 14:39:17.175786+00,2020-09-23 14:39:28.054075+00,46731,914220296,1624113.0
4,1569254,1.0,2.0,130.0,2020-09-23 14:39:57.24919+00,2020-09-23 14:39:58.963727+00,49571,1171473137,1624129.0


In [108]:
# Создать рекомендательную систему для каждого пользователя

## Создать рекомендательную систему для каждого пользователя

In [109]:
#meme['is_broken'].value_counts()

meme = meme[meme['is_broken'] == False]

In [110]:
meme

,meme_id,meme_type,original_post_link,source_name,published_at,telegram_file_id,content_url,caption,content_lang,is_available,likes,dislikes,created_at,updated_at,is_broken,source_id,content_text,caption_lang,language_id
0,13395,photo,https://vk.com/saintbeobanka?w=wall-67185996_2...,https://vk.com/saintbeobanka,2020-03-28 00:22:26+00,AgACAgQAAx0EVqG9GwACyDJefsuwVgIydYSqXSt9L2-o3Y...,https://sun6-14.userapi.com/c7004/v7004381/823...,NaN,ru,True,28,38,2020-03-28 03:59:44.496141+00,2021-02-10 07:21:58.272992+00,False,10.0,Будильник Отложить на 10 мимут vk.com/jaimes_h...,ww,2.0
1,13952,photo,https://reddit.com/r/wholesomememes/comments/f...,https://www.reddit.com/r/wholesomememes/,2020-03-28 12:14:46+00,AgACAgQAAx0EVqG9GwAC0QZef4L5HUrdey1K4xOBNC0V0-...,https://i.redd.it/xo8ilxurnep41.png,Wholesome slavery?,en,True,9,16,2020-03-28 17:01:45.289559+00,2021-02-10 07:21:58.276687+00,False,80.0,Happy Thanksgiving ! I was gonna say Let's say...,en,3.0
2,15593,photo,https://t.me/LaQeque/41396,https://t.me/LaQeque,2020-03-30 09:11:04+00,AgACAgIAAx0CVqG9GwAC7X5egk_f1FcUOOXdwvQ6tCUBOP...,https://storage.googleapis.com/meme_imgs/15593...,NaN,ru,True,41,54,2020-03-30 12:02:40.780487+00,2021-02-10 07:21:58.28043+00,False,19.0,"Ггграждане, мемння ззовут вваш преззидент, ппо...",ww,2.0
3,18442,photo,https://reddit.com/r/dankmemes/comments/ftalgn...,https://www.reddit.com/r/dankmemes/,2020-04-01 22:53:31+00,AgACAgQAAx0EVqG9GwABAQ8_XoVyYdS-2R9bcIpdH4YqK-...,https://i.redd.it/a3mp5spddaq41.jpg,You Won’t last 10 seconds,en,True,87,48,2020-04-02 05:04:33.300689+00,2021-02-10 07:21:58.28401+00,False,78.0,AL ALL WT Should we contact the hot milfs in o...,en,3.0
4,20798,photo,https://vk.com/wtf.rasha?w=wall-23486067_179559,https://vk.com/wtf.rasha,2020-04-04 11:15:01+00,AgACAgQAAx0EVqG9GwABATuvXonKXWZuSeRqN8vd8gVR09...,https://sun9-37.userapi.com/c858228/v858228646...,NaN,ru,True,234,178,2020-04-04 12:10:28.173755+00,2021-02-10 07:21:58.291519+00,False,9.0,Шляпа Патриарха Никона @NikonaShapka апрель 20...,ww,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88914,50179,photo,https://vk.com/socialburr?w=wall-97042401_85287,https://vk.com/socialburr,2020-07-12 12:14:20+00,AgACAgQAAx0EVqG9GwABAiShXwykd6_u2zHTEWqDiQz0ww...,https://sun9-40.userapi.com/c858528/v858528708...,NaN,ru,True,662,624,2020-07-13 18:14:15.455631+00,2021-02-10 07:21:58.260401+00,False,46.0,"Мы не употребляем мясо, лактозу, глютен, что н...",ww,2.0
88915,61616,photo,https://t.me/leninstreet/32006,https://t.me/leninstreet,2020-09-30 16:00:06+00,AgACAgIAAx0CVqG9GwABAoI7X3Wsk9iBIy5pcghNOaYewn...,https://storage.googleapis.com/meme_imgs/61616...,NaN,ru,True,133,150,2020-10-01 06:19:14.480151+00,2021-02-10 07:21:58.210773+00,False,29.0,решил завести собаку 15:06 Я конечно не собака...,ww,2.0
88916,22283,photo,https://vk.com/eternalclassic?w=wall-129440544...,https://vk.com/eternalclassic,2020-04-05 16:40:00+00,AgACAgQAAx0EVqG9GwABAUAPXoo6K5WekQRumii_kbBGMq...,https://sun6-14.userapi.com/c543105/v543105248...,,en,False,5,11,2020-04-05 20:06:03.525906+00,2021-02-10 07:21:58.235317+00,False,4.0,Girlfriend GF Is your gf She's your gf She's y...,und,3.0
88917,80056,photo,https://t.me/low_content/9783,https://t.me/low_content,2020-12-31 05:03:01+00,AgACAgIAAx0CVqG9GwABAvxtX_EXEDRo0nYTids0Gq_X0A...,https://storage.googleapis.com/meme_imgs/80056...,NaN,ww,True,35,54,2021-01-03 01:00:01.165269+00,2021-02-10 07:21:58.239564+00,False,147.0,NaN,ww,2.0


In [111]:
user['is_banned'].value_counts()

False    28019
Name: is_banned, dtype: int64

In [112]:
# Проверить гипотезу: англ пользователи более активные чем русские
# лайки
# дизлайки
# отношение L/D
# Диаграмма (рус в среднем и eng в среднем)

In [113]:
user['interface_lang'].value_counts()

ru    17662
en     9884
in      262
uk       34
pt       22
Name: interface_lang, dtype: int64

In [114]:
reactions = event.groupby('user_id').count()['event_id']
reactions

user_id
1615           61
3859          352
7530           13
12199           9
13934           6
             ... 
1669487431     86
1673757980    481
1674375882     36
1679813411    293
1693249754    326
Name: event_id, Length: 28064, dtype: int64

In [115]:
# 1 - лайк
# 2 - дизлайк
likes = event[event['reaction_id'] == 1].groupby('user_id').count()['event_id']
#likes.rename(columns = {'event_id':'likes'})
likes


user_id
1615           31
3859          209
7530            1
13934           1
14229           2
             ... 
1669487431     81
1673757980    218
1674375882     28
1679813411    157
1693249754     59
Name: event_id, Length: 23004, dtype: int64

In [116]:
dislikes =  event[event['reaction_id'] == 2].groupby('user_id').count()['event_id']
dislikes


user_id
1615           13
3859           93
7530            2
14229          10
16060         111
             ... 
1669487431      4
1673757980    260
1674375882      6
1679813411    124
1693249754    265
Name: event_id, Length: 21750, dtype: int64

In [117]:
event_reactions = pd.DataFrame([likes.values,dislikes.values, reactions.values]).transpose()
event_reactions.rename(columns = {0:'likes', 1:'dislikes', 2:'reactions'}, inplace = True)


In [118]:
event_reactions.index = reactions.index
event_reactions

,likes,dislikes,reactions
user_id,,,
1615,31.0,13.0,61.0
3859,209.0,93.0,352.0
7530,1.0,2.0,13.0
12199,1.0,10.0,9.0
13934,2.0,111.0,6.0
...,...,...,...
1669487431,NaN,NaN,86.0
1673757980,NaN,NaN,481.0
1674375882,NaN,NaN,36.0


### Построить график для лайков

In [119]:
likes_users_ids = likes.index
likes_users_ids

Int64Index([      1615,       3859,       7530,      13934,      14229,
                 16060,      20535,      26961,      28327,      30767,
            ...
            1614671434, 1621350758, 1633384775, 1643656152, 1658592072,
            1669487431, 1673757980, 1674375882, 1679813411, 1693249754],
           dtype='int64', name='user_id', length=23004)

In [120]:
users_likes = user[user.user_id.isin(likes_users_ids)].set_index('user_id')['interface_lang'].copy()
users_likes

user_id
1235209430    en
187967374     ru
923016092     en
69861816      ru
664894269     ru
              ..
286288293     ru
535976045     ru
25657063      ru
210046223     ru
715409940     ru
Name: interface_lang, Length: 22778, dtype: object

In [121]:
detailed_likes =  pd.DataFrame([users_likes, likes]).transpose()
grouped_det_likes = detailed_likes.groupby('interface_lang').count()
grouped_det_likes

,event_id
interface_lang,
en,7285
in,229
pt,20
ru,15157
uk,32


In [122]:
import plotly.express as px
fig = px.bar(grouped_det_likes, x = grouped_det_likes.index, y = 'event_id')
fig.show()

### Для каждого source соотношение лайков дизлайков

In [123]:
memes_grouped = meme[['source_id', 'likes', 'dislikes']]
memes_grouped_sum = memes_grouped.groupby('source_id').sum()
memes_grouped_sum['l\\d'] = memes_grouped_sum['likes'] / memes_grouped_sum['dislikes']
memes_grouped_sum

,likes,dislikes,l\d
source_id,,,
1.0,294,920,0.319565
2.0,39219,40506,0.968227
3.0,41454,48666,0.851806
4.0,29317,51699,0.567071
5.0,46651,39558,1.179306
...,...,...,...
212.0,1015,1990,0.510050
213.0,31,117,0.264957
214.0,85,269,0.315985


### Проделать ту же работу для дизлайков и всех реакций

#### Дизлайки

In [124]:
dislikes_users_ids = dislikes.index
dislikes_users_ids

Int64Index([      1615,       3859,       7530,      14229,      16060,
                 20535,      26961,      28327,      30767,      37459,
            ...
            1621350758, 1628163852, 1633384775, 1643656152, 1658592072,
            1669487431, 1673757980, 1674375882, 1679813411, 1693249754],
           dtype='int64', name='user_id', length=21750)

In [125]:
users_dislikes = user[user.user_id.isin(dislikes_users_ids)].set_index('user_id')['interface_lang'].copy()
users_dislikes

user_id
1235209430    en
187967374     ru
923016092     en
69861816      ru
664894269     ru
              ..
286288293     ru
535976045     ru
25657063      ru
210046223     ru
715409940     ru
Name: interface_lang, Length: 21513, dtype: object

In [126]:
detailed_dislikes =  pd.DataFrame([users_dislikes, dislikes]).transpose()
grouped_det_dislikes = detailed_dislikes.groupby('interface_lang').count()
grouped_det_dislikes

,event_id
interface_lang,
en,5839
in,190
pt,17
ru,15404
uk,31


In [127]:
fig = px.bar(grouped_det_dislikes, x = grouped_det_dislikes.index, y = 'event_id')
fig.show()

### Все реакции

In [128]:
reactions_users_ids = reactions.index
reactions_users_ids

Int64Index([      1615,       3859,       7530,      12199,      13934,
                 14229,      16060,      20535,      26961,      28327,
            ...
            1628163852, 1633384775, 1641681912, 1643656152, 1658592072,
            1669487431, 1673757980, 1674375882, 1679813411, 1693249754],
           dtype='int64', name='user_id', length=28064)

In [129]:
users_reactions = user[user.user_id.isin(reactions_users_ids)].set_index('user_id')['interface_lang'].copy()
users_reactions

user_id
1235209430    en
113584040     ru
1142713343    en
187967374     ru
923016092     en
              ..
286288293     ru
535976045     ru
25657063      ru
210046223     ru
715409940     ru
Name: interface_lang, Length: 27758, dtype: object

In [130]:
detailed_reactions = pd.DataFrame([users_reactions, reactions]).transpose()
grouped_det_reactions = detailed_reactions.groupby('interface_lang').count()

In [131]:
fig = px.bar(grouped_det_reactions, x = grouped_det_reactions.index, y = 'event_id')
fig.show()

In [132]:
grouped_det_likes

,event_id
interface_lang,
en,7285
in,229
pt,20
ru,15157
uk,32


In [133]:
grouped_det_dislikes

,event_id
interface_lang,
en,5839
in,190
pt,17
ru,15404
uk,31


In [134]:
grouped_det_likes / grouped_det_dislikes

,event_id
interface_lang,
en,1.247645
in,1.205263
pt,1.176471
ru,0.983965
uk,1.032258


In [135]:
grouped_det_reactions

,event_id
interface_lang,
en,9851
in,262
pt,22
ru,17524
uk,34


Выводы, которые можно сделать из проделанной работы из имеющихся данных:
- В наблюдении кол-во людей с RU интерфейсом преобладает над иными значениями в ~3 раза, но это не означает, что они более активны
- Люди с EN интерфейсом предпочитают лайкать мемы (L/D показатель = 1.28 для EN и ~0.98 для RU)

# ДЗ
- Построить гипотезы (2-3 штуки)
- Датасет + модель (какой мем какому пользователю зайдет)
- Реком система в цикле(как вариант) будет смотреть каждый мем и классифицировать на 1 или 0
- Презентация

In [136]:
meme.meme_type.value_counts()

photo        79066
video         6023
animation     3467
Name: meme_type, dtype: int64

## Гипотеза №1: отношение l/d у мужчин ниже, чем у женщин

In [137]:
data_for_gender = event[['reaction_id', 'user_id']].copy()

In [138]:
data_for_gender

,reaction_id,user_id
0,1.0,1153946021
1,2.0,285739883
2,1.0,751332478
3,1.0,914220296
4,1.0,1171473137
...,...,...
9958106,2.0,384165988
9958107,1.0,408619506
9958108,2.0,527989600
9958109,1.0,213536739


In [139]:
users = user[['user_id', 'gender']].copy()

In [140]:
users

,user_id,gender
0,1235209430,NaN
1,113584040,N
2,1142713343,NaN
3,187967374,M
4,923016092,NaN
...,...,...
28014,286288293,NaN
28015,535976045,M
28016,25657063,M
28017,210046223,NaN


In [141]:
users['gender'].value_counts()

M    2865
N    2025
F    1161
Name: gender, dtype: int64

In [142]:
inner = pd.merge(users, data_for_gender, on='user_id')

In [143]:
users_likes_w_gender = inner[inner['reaction_id'] == 1].copy()
users_dislikes_w_gender = inner[inner['reaction_id'] == 2].copy()


In [144]:
users_likes_w_gender

,user_id,gender,reaction_id
0,1235209430,NaN,1.0
1,1235209430,NaN,1.0
12,187967374,M,1.0
20,187967374,M,1.0
29,923016092,NaN,1.0
...,...,...,...
9942980,715409940,NaN,1.0
9942982,715409940,NaN,1.0
9942983,715409940,NaN,1.0
9942984,715409940,NaN,1.0


In [145]:
M_likes = users_likes_w_gender[users_likes_w_gender['gender'] == 'M'].copy()
F_likes = users_likes_w_gender[users_likes_w_gender['gender'] == 'F'].copy()
M_dislikes = users_dislikes_w_gender[users_dislikes_w_gender['gender'] == 'M'].copy()
F_dislikes = users_dislikes_w_gender[users_dislikes_w_gender['gender'] == 'F'].copy()

In [146]:
MF_values_dict = {'M/F_ratio':[len(M_likes) / len (M_dislikes), len(F_likes) / len (F_dislikes)]}
MF_values = pd.DataFrame(MF_values_dict)
MF_values.rename(index={0:'M',1:'F'}, inplace = True)
MF_values

,M/F_ratio
M,1.057647
F,1.501370


Как и предполагалось, L/D отношение у мужчин ниже, чем у женщин. Возможно, это связано с тем, что мужчины в целом как юзеры более активны чем женщины. Возможно, мужчины в Интернете более категоричны и при недовольстве чаще ставят дизлайк, когда представительницы женского пола просто проигнорируют и пойдут дальше  

## Гипотеза №2: фото мемы в среднем лайкают больше, чем мемы иного типа

In [147]:
meme.meme_type.value_counts()

photo        79066
video         6023
animation     3467
Name: meme_type, dtype: int64

In [148]:
detailed_meme = meme[['meme_type', 'likes', 'dislikes']].copy()
detailed_meme.groupby('meme_type').mean()

,likes,dislikes
meme_type,,
animation,24.116527,33.526392
photo,54.688134,51.532479
video,14.654325,26.687033


Как и предполагалось, в среднем из 3 типов мемов, большее кол-во лайков собирают мемы-картинки, а не мемы в виде видео или анимации. Это можно объяснить тем, что юзеру удобнее(быстрее) посмотреть картинку, и ему не нужно тратить много времени на ее просмотр, поэтому и лайк проще получить картинке. 

## Датасет + модель (какой мем какому пользователю зайдет)

* Подготовка данных

In [149]:
event.head()

,event_id,reaction_id,session_idx,meme_idx,created_at,reacted_at,meme_id,user_id,telegram_message_id
0,1569409,1.0,1.0,171.0,2020-09-23 14:45:40.204598+00,2020-09-23 14:45:44.079206+00,9197,1153946021,1624299.0
1,1569228,2.0,199.0,189.0,2020-09-23 14:38:56.25774+00,2020-09-23 14:39:11.978709+00,34459,285739883,1624096.0
2,1569405,1.0,7.0,133.0,2020-09-23 14:45:34.244269+00,2020-09-23 14:45:46.699976+00,10879,751332478,1624295.0
3,1569239,1.0,2.0,265.0,2020-09-23 14:39:17.175786+00,2020-09-23 14:39:28.054075+00,46731,914220296,1624113.0
4,1569254,1.0,2.0,130.0,2020-09-23 14:39:57.24919+00,2020-09-23 14:39:58.963727+00,49571,1171473137,1624129.0


In [150]:
meme.head()

,meme_id,meme_type,original_post_link,source_name,published_at,telegram_file_id,content_url,caption,content_lang,is_available,likes,dislikes,created_at,updated_at,is_broken,source_id,content_text,caption_lang,language_id
0,13395,photo,https://vk.com/saintbeobanka?w=wall-67185996_2...,https://vk.com/saintbeobanka,2020-03-28 00:22:26+00,AgACAgQAAx0EVqG9GwACyDJefsuwVgIydYSqXSt9L2-o3Y...,https://sun6-14.userapi.com/c7004/v7004381/823...,NaN,ru,True,28,38,2020-03-28 03:59:44.496141+00,2021-02-10 07:21:58.272992+00,False,10.0,Будильник Отложить на 10 мимут vk.com/jaimes_h...,ww,2.0
1,13952,photo,https://reddit.com/r/wholesomememes/comments/f...,https://www.reddit.com/r/wholesomememes/,2020-03-28 12:14:46+00,AgACAgQAAx0EVqG9GwAC0QZef4L5HUrdey1K4xOBNC0V0-...,https://i.redd.it/xo8ilxurnep41.png,Wholesome slavery?,en,True,9,16,2020-03-28 17:01:45.289559+00,2021-02-10 07:21:58.276687+00,False,80.0,Happy Thanksgiving ! I was gonna say Let's say...,en,3.0
2,15593,photo,https://t.me/LaQeque/41396,https://t.me/LaQeque,2020-03-30 09:11:04+00,AgACAgIAAx0CVqG9GwAC7X5egk_f1FcUOOXdwvQ6tCUBOP...,https://storage.googleapis.com/meme_imgs/15593...,NaN,ru,True,41,54,2020-03-30 12:02:40.780487+00,2021-02-10 07:21:58.28043+00,False,19.0,"Ггграждане, мемння ззовут вваш преззидент, ппо...",ww,2.0
3,18442,photo,https://reddit.com/r/dankmemes/comments/ftalgn...,https://www.reddit.com/r/dankmemes/,2020-04-01 22:53:31+00,AgACAgQAAx0EVqG9GwABAQ8_XoVyYdS-2R9bcIpdH4YqK-...,https://i.redd.it/a3mp5spddaq41.jpg,You Won’t last 10 seconds,en,True,87,48,2020-04-02 05:04:33.300689+00,2021-02-10 07:21:58.28401+00,False,78.0,AL ALL WT Should we contact the hot milfs in o...,en,3.0
4,20798,photo,https://vk.com/wtf.rasha?w=wall-23486067_179559,https://vk.com/wtf.rasha,2020-04-04 11:15:01+00,AgACAgQAAx0EVqG9GwABATuvXonKXWZuSeRqN8vd8gVR09...,https://sun9-37.userapi.com/c858228/v858228646...,NaN,ru,True,234,178,2020-04-04 12:10:28.173755+00,2021-02-10 07:21:58.291519+00,False,9.0,Шляпа Патриарха Никона @NikonaShapka апрель 20...,ww,2.0


In [151]:
user.head()

,user_id,username,first_name,last_name,language_code,seed,deep_link,is_blocked_bot,is_banned,is_admin,is_moderator,sessions,last_session_memes,watched_memes,created_at,updated_at,interface_lang,gender,latest_meme_sent_at
0,1235209430,NaN,Paras,Kamboj,NaN,6,likemeupbot_lang_en,True,False,False,0,3,4,8,2020-10-27 07:33:12.040495+00,2021-02-10 00:29:57.669801+00,en,NaN,NaN
1,113584040,nikrnrr,N,NaN,ru,26,NaN,True,False,False,0,1,1,0,2020-10-28 12:05:37.473558+00,2021-02-10 01:29:57.505289+00,ru,N,NaN
2,1142713343,NaN,Serik,NaN,NaN,32,NaN,True,False,False,0,1,1,0,2020-10-27 17:31:30.480084+00,2021-02-10 01:29:55.555919+00,en,NaN,NaN
3,187967374,Ddivanov,Dmitry,Ivanov,ru,13,NaN,True,False,False,0,2,5,9,2020-10-26 13:12:52.373779+00,2021-02-09 23:29:56.086916+00,ru,M,NaN
4,923016092,NaN,Joanna,NaN,es,99,NaN,False,False,False,0,39,1,35,2020-10-27 19:11:33.959974+00,2021-02-09 17:56:04.574689+00,en,NaN,2021-02-09 17:56:04.574558+00


In [154]:
# Создал копии датасетов, чтобы не изменять начальные файлы
event_df = event.copy()
meme_df = meme.copy()
user_df = user.copy()

In [155]:
#Удаление данных, которые неинформативны в рамках исследования
event_df.drop(['session_idx', 'meme_idx', 'created_at', 'reacted_at', 'event_id'], axis = 1, inplace = True)

In [156]:
meme_df.columns

Index(['meme_id', 'meme_type', 'original_post_link', 'source_name',
       'published_at', 'telegram_file_id', 'content_url', 'caption',
       'content_lang', 'is_available', 'likes', 'dislikes', 'created_at',
       'updated_at', 'is_broken', 'source_id', 'content_text', 'caption_lang',
       'language_id'],
      dtype='object')

In [157]:
#Удаление данных, которые неинформативны в рамках исследования
meme_df.drop(['published_at', 'caption','created_at', 'updated_at','content_text'], axis = 1, inplace = True)

In [158]:
user_df.columns

Index(['user_id', 'username', 'first_name', 'last_name', 'language_code',
       'seed', 'deep_link', 'is_blocked_bot', 'is_banned', 'is_admin',
       'is_moderator', 'sessions', 'last_session_memes', 'watched_memes',
       'created_at', 'updated_at', 'interface_lang', 'gender',
       'latest_meme_sent_at'],
      dtype='object')

### Работа с user_df

In [159]:
#Удаление данных, которые неинформативны в рамках исследования (тут хотел бы отметить, что поле gender не хотелось удалять, но по нему нет данных по большинству записей, поэтому был вынужден удалить это поле)
user_df.drop(['deep_link','last_session_memes', 'created_at', 'updated_at', 'is_banned','is_moderator', 'gender', 'is_blocked_bot'], axis = 1, inplace = True)

In [160]:
user_df.interface_lang.replace({'en':1, 'ru':0}, inplace = True) # 1 если интерфейс английский, 0 если русский

In [161]:
user_df.interface_lang.value_counts() # ~75% используют русский интерфейс

0     17662
1      9884
in      262
uk       34
pt       22
Name: interface_lang, dtype: int64

In [162]:
user_df.isna().sum()

user_id                    0
username                7785
first_name                 2
last_name              13458
language_code           6897
seed                       0
is_admin                   0
sessions                   0
watched_memes              0
interface_lang           155
latest_meme_sent_at    11958
dtype: int64

In [163]:
# Конечно, 48 nan можно было запихнуть в любую категорию, т.к. малое число пропусков, но захотел распределить пропорционально
import random
nans = user_df['interface_lang'].isna()
length = sum(nans)
replacement = random.choices([0, 1], weights=[.75, .25], k=length)
user_df.loc[nans,'interface_lang'] = replacement
user_df.isna().sum()

user_id                    0
username                7785
first_name                 2
last_name              13458
language_code           6897
seed                       0
is_admin                   0
sessions                   0
watched_memes              0
interface_lang             0
latest_meme_sent_at    11958
dtype: int64

In [164]:
user_df.interface_lang.value_counts() # Не получилось :)

0     17778
1      9923
in      262
uk       34
pt       22
Name: interface_lang, dtype: int64

In [165]:
user_df.head() # отформатированный датасэт

,user_id,username,first_name,last_name,language_code,seed,is_admin,sessions,watched_memes,interface_lang,latest_meme_sent_at
0,1235209430,NaN,Paras,Kamboj,NaN,6,False,3,8,1,NaN
1,113584040,nikrnrr,N,NaN,ru,26,False,1,0,0,NaN
2,1142713343,NaN,Serik,NaN,NaN,32,False,1,0,1,NaN
3,187967374,Ddivanov,Dmitry,Ivanov,ru,13,False,2,9,0,NaN
4,923016092,NaN,Joanna,NaN,es,99,False,39,35,1,2021-02-09 17:56:04.574558+00


### Работа с event_df

In [166]:
event_df.head()

,reaction_id,meme_id,user_id,telegram_message_id
0,1.0,9197,1153946021,1624299.0
1,2.0,34459,285739883,1624096.0
2,1.0,10879,751332478,1624295.0
3,1.0,46731,914220296,1624113.0
4,1.0,49571,1171473137,1624129.0


In [167]:
event_df = event_df.drop(event_df[(event_df.reaction_id == 3) | (event_df.reaction_id == 4) |(event_df.reaction_id == 5) ].index) # Удаление действий админов

In [168]:
event_df.isna().sum()

reaction_id            631914
meme_id                     0
user_id                     0
telegram_message_id    290824
dtype: int64

In [169]:
event_df.reaction_id.value_counts()

1.0    4645144
2.0    4538667
Name: reaction_id, dtype: int64

In [170]:
nans = event_df['reaction_id'].isna()  # пропорциональное распределение лайков/дизлайков по nan значениям
length = sum(nans)
replacement = random.choices([1, 2], weights=[.5, .5], k=length)
event_df.loc[nans,'reaction_id'] = replacement
event_df.isna().sum()

reaction_id                 0
meme_id                     0
user_id                     0
telegram_message_id    290824
dtype: int64

In [171]:
event_df.reaction_id.value_counts()

1.0    4960930
2.0    4854795
Name: reaction_id, dtype: int64

In [172]:
event_df.head() # отформатированный датасэт

,reaction_id,meme_id,user_id,telegram_message_id
0,1.0,9197,1153946021,1624299.0
1,2.0,34459,285739883,1624096.0
2,1.0,10879,751332478,1624295.0
3,1.0,46731,914220296,1624113.0
4,1.0,49571,1171473137,1624129.0


### Работа с meme_df

In [173]:
meme_df.head()

,meme_id,meme_type,original_post_link,source_name,telegram_file_id,content_url,content_lang,is_available,likes,dislikes,is_broken,source_id,caption_lang,language_id
0,13395,photo,https://vk.com/saintbeobanka?w=wall-67185996_2...,https://vk.com/saintbeobanka,AgACAgQAAx0EVqG9GwACyDJefsuwVgIydYSqXSt9L2-o3Y...,https://sun6-14.userapi.com/c7004/v7004381/823...,ru,True,28,38,False,10.0,ww,2.0
1,13952,photo,https://reddit.com/r/wholesomememes/comments/f...,https://www.reddit.com/r/wholesomememes/,AgACAgQAAx0EVqG9GwAC0QZef4L5HUrdey1K4xOBNC0V0-...,https://i.redd.it/xo8ilxurnep41.png,en,True,9,16,False,80.0,en,3.0
2,15593,photo,https://t.me/LaQeque/41396,https://t.me/LaQeque,AgACAgIAAx0CVqG9GwAC7X5egk_f1FcUOOXdwvQ6tCUBOP...,https://storage.googleapis.com/meme_imgs/15593...,ru,True,41,54,False,19.0,ww,2.0
3,18442,photo,https://reddit.com/r/dankmemes/comments/ftalgn...,https://www.reddit.com/r/dankmemes/,AgACAgQAAx0EVqG9GwABAQ8_XoVyYdS-2R9bcIpdH4YqK-...,https://i.redd.it/a3mp5spddaq41.jpg,en,True,87,48,False,78.0,en,3.0
4,20798,photo,https://vk.com/wtf.rasha?w=wall-23486067_179559,https://vk.com/wtf.rasha,AgACAgQAAx0EVqG9GwABATuvXonKXWZuSeRqN8vd8gVR09...,https://sun9-37.userapi.com/c858228/v858228646...,ru,True,234,178,False,9.0,ww,2.0


In [174]:
meme_df.drop(['content_lang', 'caption_lang', 'is_broken'], axis = 1, inplace = True)
meme_df.is_available.value_counts()

True     74594
False    13962
Name: is_available, dtype: int64

In [175]:
meme_df.isna().sum()

meme_id                  0
meme_type                0
original_post_link    1100
source_name              0
telegram_file_id        29
content_url           8829
is_available             0
likes                    0
dislikes                 0
source_id             2399
language_id           1770
dtype: int64

In [176]:
meme_df = pd.get_dummies(meme_df, columns = ['meme_type'])

In [177]:
result = meme_df['is_available'].astype(int)
meme_df['is_available'] = result
del result

In [178]:
meme_df.source_id.value_counts()

75.0     5362
78.0     4207
76.0     2884
80.0     2808
4.0      2691
         ... 
101.0      10
178.0       9
213.0       7
107.0       3
210.0       2
Name: source_id, Length: 193, dtype: int64

In [179]:
nans = meme_df['source_id'].isna() # пропорциональное распределение источников по nan значениям
source_ids = meme_df.source_id.unique()
source_ids = source_ids[np.isnan(source_ids) == False]
length = sum(nans)
replacement = random.choices(source_ids, k=length)
meme_df.loc[nans,'source_id'] = replacement

In [180]:
meme_df.source_id.value_counts()

75.0     5375
78.0     4211
76.0     2895
80.0     2819
4.0      2704
         ... 
85.0       20
107.0      17
178.0      16
213.0      12
210.0      11
Name: source_id, Length: 193, dtype: int64

In [181]:
meme_df.isna().sum()

meme_id                   0
original_post_link     1100
source_name               0
telegram_file_id         29
content_url            8829
is_available              0
likes                     0
dislikes                  0
source_id                 0
language_id            1770
meme_type_animation       0
meme_type_photo           0
meme_type_video           0
dtype: int64

In [182]:
meme_df.language_id.value_counts()

2.0    41608
3.0    36599
4.0     5151
1.0     3172
5.0      256
Name: language_id, dtype: int64

In [183]:
nans = meme_df['language_id'].isna() # пропорциональное распределение языков по nan значениям
language_ids = meme_df.language_id.unique()
language_ids = language_ids[np.isnan(language_ids) == False]
length = sum(nans)
replacement = random.choices(language_ids, k=length)
meme_df.loc[nans,'language_id'] = replacement

In [184]:
meme_df.language_id.value_counts()

2.0    41937
3.0    36974
4.0     5514
1.0     3528
5.0      603
Name: language_id, dtype: int64

In [185]:
meme_df.isna().sum()

meme_id                   0
original_post_link     1100
source_name               0
telegram_file_id         29
content_url            8829
is_available              0
likes                     0
dislikes                  0
source_id                 0
language_id               0
meme_type_animation       0
meme_type_photo           0
meme_type_video           0
dtype: int64

In [186]:
meme_df.head() # готовый датафрейм

,meme_id,original_post_link,source_name,telegram_file_id,content_url,is_available,likes,dislikes,source_id,language_id,meme_type_animation,meme_type_photo,meme_type_video
0,13395,https://vk.com/saintbeobanka?w=wall-67185996_2...,https://vk.com/saintbeobanka,AgACAgQAAx0EVqG9GwACyDJefsuwVgIydYSqXSt9L2-o3Y...,https://sun6-14.userapi.com/c7004/v7004381/823...,1,28,38,10.0,2.0,0,1,0
1,13952,https://reddit.com/r/wholesomememes/comments/f...,https://www.reddit.com/r/wholesomememes/,AgACAgQAAx0EVqG9GwAC0QZef4L5HUrdey1K4xOBNC0V0-...,https://i.redd.it/xo8ilxurnep41.png,1,9,16,80.0,3.0,0,1,0
2,15593,https://t.me/LaQeque/41396,https://t.me/LaQeque,AgACAgIAAx0CVqG9GwAC7X5egk_f1FcUOOXdwvQ6tCUBOP...,https://storage.googleapis.com/meme_imgs/15593...,1,41,54,19.0,2.0,0,1,0
3,18442,https://reddit.com/r/dankmemes/comments/ftalgn...,https://www.reddit.com/r/dankmemes/,AgACAgQAAx0EVqG9GwABAQ8_XoVyYdS-2R9bcIpdH4YqK-...,https://i.redd.it/a3mp5spddaq41.jpg,1,87,48,78.0,3.0,0,1,0
4,20798,https://vk.com/wtf.rasha?w=wall-23486067_179559,https://vk.com/wtf.rasha,AgACAgQAAx0EVqG9GwABATuvXonKXWZuSeRqN8vd8gVR09...,https://sun9-37.userapi.com/c858228/v858228646...,1,234,178,9.0,2.0,0,1,0


# Работа над задачей

## Топ 100 мемов (09.03.21) 

In [187]:
event_df

,reaction_id,meme_id,user_id,telegram_message_id
0,1.0,9197,1153946021,1624299.0
1,2.0,34459,285739883,1624096.0
2,1.0,10879,751332478,1624295.0
3,1.0,46731,914220296,1624113.0
4,1.0,49571,1171473137,1624129.0
...,...,...,...,...
9958106,2.0,54516,384165988,2845009.0
9958107,1.0,14263,408619506,2845055.0
9958108,2.0,46211,527989600,2845046.0
9958109,1.0,30231,213536739,2845068.0


In [196]:
new_meme_df = meme_df.copy()
new_meme_df = new_meme_df[(new_meme_df['likes'] + new_meme_df['dislikes']) > 20]
new_meme_df['l/d'] = new_meme_df['likes'] / new_meme_df['dislikes']

In [200]:
top_100_memes = new_meme_df.groupby(['meme_id']).mean().sort_values(by = 'l/d', ascending=False)[0:100]

In [225]:
bot_100_memes = new_meme_df.groupby(['meme_id']).mean().sort_values(by = 'l/d', ascending=False)[-100:]

In [203]:
top_100_memes

,is_available,likes,dislikes,source_id,language_id,meme_type_animation,meme_type_photo,meme_type_video,l/d
meme_id,,,,,,,,,
28269,0,225,81,5.0,3.0,0,1,0,2.777778
45814,0,263,96,5.0,3.0,0,1,0,2.739583
38443,0,353,129,5.0,3.0,0,1,0,2.736434
55032,0,336,123,5.0,3.0,0,1,0,2.731707
19809,0,286,105,5.0,3.0,0,1,0,2.723810
...,...,...,...,...,...,...,...,...,...
22460,0,139,53,5.0,3.0,0,1,0,2.622642
28087,0,118,45,5.0,3.0,0,1,0,2.622222
12869,0,118,45,5.0,3.0,0,1,0,2.622222


In [206]:
meme.set_index('meme_id', inplace = True)

In [208]:
meme

,meme_type,original_post_link,source_name,published_at,telegram_file_id,content_url,caption,content_lang,is_available,likes,dislikes,created_at,updated_at,is_broken,source_id,content_text,caption_lang,language_id
meme_id,,,,,,,,,,,,,,,,,,
13395,photo,https://vk.com/saintbeobanka?w=wall-67185996_2...,https://vk.com/saintbeobanka,2020-03-28 00:22:26+00,AgACAgQAAx0EVqG9GwACyDJefsuwVgIydYSqXSt9L2-o3Y...,https://sun6-14.userapi.com/c7004/v7004381/823...,NaN,ru,True,28,38,2020-03-28 03:59:44.496141+00,2021-02-10 07:21:58.272992+00,False,10.0,Будильник Отложить на 10 мимут vk.com/jaimes_h...,ww,2.0
13952,photo,https://reddit.com/r/wholesomememes/comments/f...,https://www.reddit.com/r/wholesomememes/,2020-03-28 12:14:46+00,AgACAgQAAx0EVqG9GwAC0QZef4L5HUrdey1K4xOBNC0V0-...,https://i.redd.it/xo8ilxurnep41.png,Wholesome slavery?,en,True,9,16,2020-03-28 17:01:45.289559+00,2021-02-10 07:21:58.276687+00,False,80.0,Happy Thanksgiving ! I was gonna say Let's say...,en,3.0
15593,photo,https://t.me/LaQeque/41396,https://t.me/LaQeque,2020-03-30 09:11:04+00,AgACAgIAAx0CVqG9GwAC7X5egk_f1FcUOOXdwvQ6tCUBOP...,https://storage.googleapis.com/meme_imgs/15593...,NaN,ru,True,41,54,2020-03-30 12:02:40.780487+00,2021-02-10 07:21:58.28043+00,False,19.0,"Ггграждане, мемння ззовут вваш преззидент, ппо...",ww,2.0
18442,photo,https://reddit.com/r/dankmemes/comments/ftalgn...,https://www.reddit.com/r/dankmemes/,2020-04-01 22:53:31+00,AgACAgQAAx0EVqG9GwABAQ8_XoVyYdS-2R9bcIpdH4YqK-...,https://i.redd.it/a3mp5spddaq41.jpg,You Won’t last 10 seconds,en,True,87,48,2020-04-02 05:04:33.300689+00,2021-02-10 07:21:58.28401+00,False,78.0,AL ALL WT Should we contact the hot milfs in o...,en,3.0
20798,photo,https://vk.com/wtf.rasha?w=wall-23486067_179559,https://vk.com/wtf.rasha,2020-04-04 11:15:01+00,AgACAgQAAx0EVqG9GwABATuvXonKXWZuSeRqN8vd8gVR09...,https://sun9-37.userapi.com/c858228/v858228646...,NaN,ru,True,234,178,2020-04-04 12:10:28.173755+00,2021-02-10 07:21:58.291519+00,False,9.0,Шляпа Патриарха Никона @NikonaShapka апрель 20...,ww,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50179,photo,https://vk.com/socialburr?w=wall-97042401_85287,https://vk.com/socialburr,2020-07-12 12:14:20+00,AgACAgQAAx0EVqG9GwABAiShXwykd6_u2zHTEWqDiQz0ww...,https://sun9-40.userapi.com/c858528/v858528708...,NaN,ru,True,662,624,2020-07-13 18:14:15.455631+00,2021-02-10 07:21:58.260401+00,False,46.0,"Мы не употребляем мясо, лактозу, глютен, что н...",ww,2.0
61616,photo,https://t.me/leninstreet/32006,https://t.me/leninstreet,2020-09-30 16:00:06+00,AgACAgIAAx0CVqG9GwABAoI7X3Wsk9iBIy5pcghNOaYewn...,https://storage.googleapis.com/meme_imgs/61616...,NaN,ru,True,133,150,2020-10-01 06:19:14.480151+00,2021-02-10 07:21:58.210773+00,False,29.0,решил завести собаку 15:06 Я конечно не собака...,ww,2.0
22283,photo,https://vk.com/eternalclassic?w=wall-129440544...,https://vk.com/eternalclassic,2020-04-05 16:40:00+00,AgACAgQAAx0EVqG9GwABAUAPXoo6K5WekQRumii_kbBGMq...,https://sun6-14.userapi.com/c543105/v543105248...,,en,False,5,11,2020-04-05 20:06:03.525906+00,2021-02-10 07:21:58.235317+00,False,4.0,Girlfriend GF Is your gf She's your gf She's y...,und,3.0


In [213]:
pd.set_option('display.max_colwidth', -1)
pd.concat([top_100_memes, meme], axis=1, join = 'inner')[['l/d', 'likes', 'dislikes', 'content_url']]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning:

Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.



,l/d,likes,likes,dislikes,dislikes,content_url
meme_id,,,,,,
28269,2.777778,225,225,81,81,https://sun6-19.userapi.com/T3qCfSmcCenPpL2DF0oW-uRnCi7lEizwFRUvHQ/OmBUS_TPMFU.jpg
45814,2.739583,263,263,96,96,https://sun6-16.userapi.com/GGI3IjbMC2a5M1ISnGK2X2U1McBfepq55ISk_g/IhI-LN4Ibko.jpg
38443,2.736434,353,353,129,129,https://sun6-16.userapi.com/le7Gwf3DVaaleJ0PgCYWjp9-Wy2S7YUepds4IA/bF_XURUAXJA.jpg
55032,2.731707,336,336,123,123,https://sun6-13.userapi.com/tC8FPaPZNyjsLnSleN0KpC5ZoOBNaSQ7lY6enA/U3tAHAdzyTQ.jpg
19809,2.723810,286,286,105,105,https://sun6-19.userapi.com/OzbFbfBQfJlGoUhhHOALXPEGhCMmLtyYoEG7_Q/KT-oODzW76Y.jpg
...,...,...,...,...,...,...
22460,2.622642,139,139,53,53,https://sun6-14.userapi.com/lCxCv_1dJwAjmexzCRNqrZpHrYGXMHhcQW6K6w/nUecswuGzIA.jpg
28087,2.622222,118,118,45,45,https://sun6-19.userapi.com/0UwyxbPgI52wD7EVT1xmI_0zs12DTBsJMpmsIQ/Y8-n8__1mSA.jpg
12869,2.622222,118,118,45,45,https://sun6-14.userapi.com/_ExRyiHTPB9SXV-KOz3ncCiINUqpPwxoyCM-hg/K7mWweQL-yE.jpg


In [226]:
pd.concat([bot_100_memes, meme], axis=1, join = 'inner')[['l/d', 'likes', 'dislikes', 'content_url']]

,l/d,likes,likes,dislikes,dislikes,content_url
meme_id,,,,,,
51679,0.045455,1,1,22,22,https://storage.googleapis.com/meme_imgs/51679.jpg
22090,0.045455,1,1,22,22,https://sun6-14.userapi.com/c206520/v206520277/d0085/Jt0QpzafeSI.jpg
7371,0.045455,1,1,22,22,https://storage.googleapis.com/meme_imgs/7371.jpg
35831,0.045455,1,1,22,22,https://scontent-lax3-1.xx.fbcdn.net/v/t1.0-9/fr/cp0/e15/q65/58019530_420959335370102_4119530367428329472_n.jpg?_nc_cat=109&_nc_sid=110474&efg=eyJpIjoidCJ9&_nc_ohc=8bBMRufHc4YAX-fgHKm&_nc_ht=scontent-lax3-1.xx&_nc_tp=14&oh=18aa28348133c65ddff4311d2842f128&oe=5ED486C4
12062,0.045455,1,1,22,22,NaN
...,...,...,...,...,...,...
61655,0.000000,0,0,24,24,NaN
15694,0.000000,0,0,21,21,https://i.redd.it/4auha3lxysp41.png
80765,0.000000,0,0,23,23,https://storage.googleapis.com/meme_imgs/80765.jpg
